In [1]:
%pip install pyproj
%pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 8.4 MB/s 


In [40]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from pyproj import Transformer
import folium
from folium import plugins

In [55]:
url = 'https://www.mapi.gov.il/ProfessionalInfo/Documents/dataGov/CITY.xml'
response = requests.get(url)
root = ET.fromstring(response.content)
root

<Element '{http://www.esri.com/schemas/ArcGIS/9.3}Workspace' at 0x7f93bd64f470>

In [56]:
data_geo = []
temp = []
wkt = ''

In [57]:
for x in root.iter('WKT'):
  wkt = x.text
wkt

'PROJCS["Israel_TM_Grid",GEOGCS["GCS_Israel",DATUM["D_Israel",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",219529.584],PARAMETER["False_Northing",626907.39],PARAMETER["Central_Meridian",35.20451694444445],PARAMETER["Scale_Factor",1.0000067],PARAMETER["Latitude_Of_Origin",31.73439361111111],UNIT["Meter",1.0],AUTHORITY["EPSG",2039]]'

In [58]:
transformer = Transformer.from_crs(wkt, "EPSG:4326")

In [59]:
for rec in root.findall("./WorkspaceData/DatasetData/Data/Records/Record"):
  for x in rec.itertext():
    if temp is None:
      temp = [x]
    else:
      temp = temp + [x]
  
  data_geo.append([temp[4],temp[5],temp[9],temp[1],temp[2]])
  temp = [] 


In [60]:
for rec in data_geo:
  lat, lng  = transformer.transform(rec[3], rec[4])
  rec += [lat, lng]
  

In [61]:
columns = ('semel','nameheb','nameeng','crs_x','crs_y','lat','lng')

df = pd.DataFrame(data_geo,columns=columns)
df

,semel,nameheb,nameeng,crs_x,crs_y,lat,lng
0,862,גני יוחנן,GANNE YOHANAN,184689.8424,640598.3157,31.857703,34.837087
1,863,גינתון,GINNATON,191979.3488,652283.0239,31.963276,34.913778
2,864,בקוע,BEQOA,193011.064,637420.984099999,31.829272,34.925097
3,865,שיבולים,SHIBBOLIM,162693.9971,589541.003599999,31.396378,34.607626
4,866,יטבתה,YOTVATA,205802.5761,422945.231799999,29.894997,35.063080
...,...,...,...,...,...,...,...
1235,2610,בית שמש,BET SHEMESH,199203.4669,628089.6932,31.745246,34.990701
1236,2620,קריית אונו,QIRYAT ONO,186802.4399,662891.2008,32.058802,34.858656
1237,2630,קריית גת,QIRYAT GAT,178645.1995,612555.1406,31.604603,34.774388
1238,2640,ראש העין,ROSH HAAYIN,196219.0173,667109.627800001,32.097076,34.958269


In [54]:
map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=7, control_scale=True)
for index, location_info in df.iterrows():
    folium.Marker([location_info["lat"], location_info["lng"]], 
                  popup=location_info["nameheb"]).add_to(map)
map